# Vector Approximation

In this notebook, I'm going to show how to use `qubo` to approximate a single vector.


\begin{equation}
\arg\min_b
\Vert X - 
\begin{bmatrix}
    b_{11} & b_{12} & b_{13} & \dots  & b_{1n} \\
    b_{21} & b_{22} & b_{23} & \dots  & b_{2n} \\
    \vdots & \vdots & \vdots & \ddots & \vdots \\
    b_{d1} & b_{d2} & b_{d3} & \dots  & b_{dn}
\end{bmatrix}
\cdot
\begin{bmatrix}
    2^s \\
    2^{s-1} \\
    \vdots \\
    2^{s-p}
\end{bmatrix}
\Vert
\end{equation}

The variables with $b$ prefix variables in above matrix will be $\{0, 1\}$. The equation takes matrix norm as a hamiltonian and encoded into `qubo` by using **pyqubo** module. The solver used to solve this equation is **Digital Annealing Unit** provided by Fujitsu Limited.

In [30]:
from pyqubo import Array, Binary, Model
import numpy as np
import scipy as sp
import requests
import json
import re
import time
import yaml
from lib.util import *

In [31]:
def generate_random_vector(n):
    vector = np.random.rand(n)
    return vector

In [32]:
precision = 21
mat = create_binary_variable_matrix('vectorA', (20, precision))
binary_code_mat = np.matrix(create_binary_representated_vector(precision, 1)).transpose()

approximated_vector = mat * binary_code_mat

X = np.matrix(generate_random_vector(20)).transpose()

hamiltonian = np.trace(np.inner((X - (mat * binary_code_mat)), (X - (mat * binary_code_mat))))

In [33]:
start_time = time.time()

model = hamiltonian.compile()
qubo, offset = model.to_qubo()
matrix_term = get_matrix_term(qubo, model.variables)

end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time, " seconds")

Elapsed time:  0.04717206954956055  seconds


In [7]:
problem_body = {}
DA_Solver = {}
DA_Solver["time_limit_sec"]= 100
DA_Solver["penalty_coef"]=10000
DA_Solver["num_run"] = 16
DA_Solver["num_group"] = 16
DA_Solver['gs_level'] = 50
DA_Solver['gs_cutoff'] = 80000
problem_body["fujitsuDA3"]=DA_Solver
problem_body["binary_polynomial"] = {'terms' : matrix_term}

In [8]:
response = post_solve(problem_body)
job_id = response.json()['job_id']

{'message': 'success'}


In [10]:
solution = get_solution(job_id)
configuration = solution['qubo_solution']['solutions'][0]['configuration']

In [22]:
binary_matrix = np.zeros((20, precision))
for key, value in configuration.items():
    vector_element = model.variables[int(key)]
    
    # exctract indices
    
    indices = re.findall(r'\[(\d+)\]', vector_element)
    indices = [int(_) for _ in indices]
    if value == True:
        val = 1
    else:
        val = 0
    binary_matrix[indices[0]][indices[1]] = val
binary_matrix = np.matrix(binary_matrix)

In [29]:
np.trace(np.inner((X - binary_matrix * binary_code_mat), (X - binary_matrix * binary_code_mat)))

4.287746517787633e-12